In [ ]:
# dataset load
from os.path import join
from google.colab import drive

ROOT = '/content/drive'
drive.mount(ROOT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import sys

MODULE_PATH = '/content/drive/MyDrive/GitHub/DL_Study/Base'

sys.path.insert(0, MODULE_PATH)
sys.path

['/content/drive/MyDrive/GitHub/DL_Study/Base',
 '',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [ ]:
# import
from config import *
from layers import Convolution, Pooling, FullyConnected
from optim import Adam

# for time series split
!pip install scikit-learn==0.24.2

In [ ]:
# MSE for Loss
def MSE(y, t):
    return 0.5 * np.mean((y-t)**2)

# ReLU activation function
class ReLU:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x<=0)
        y = x.copy()
        y[self.mask] = 0
        return y

    def backward(self, dy):
        dy[self.mask] = 0
        dx = dy
        return dx

# Loss with ReLU
class ReluWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.activation = ReLU()
        self.cache = None

    def forward(self, x, t):
        N, V = x.shape      # batch, output
        
        x = x.reshape(N, V)
        t = t.reshape(N, V)
        x = self.activation.forward(x)

        loss = MSE(x, t)
        self.cache = (t, x, (N, V))
        return loss

    def backward(self, dy=1):
        t, x, (N, V) = self.cache
        dx = dy * (x-t) / N
        
        dx = self.activation.backward(dx)
        dx = dx.reshape(N, V)
        return dx

In [ ]:
# CNN Model with Regression
class CnnModelReg:
    def __init__(self, input_dim = (1, 24, 8), 
                 params={'filter_num':30, 'filter_size':5, 
                         'padding':0, 'stride':1},
                 hidden_size=100, output_size=1):
        filter_num = params['filter_num']
        filter_size = params['filter_size']
        padding = params['padding']
        stride = params['stride']

        # not square
        input_size = input_dim[1]
        input_size2 = input_dim[2]
        conv_output_size_h = (input_size - filter_size + 2*padding)/stride + 1
        conv_output_size_w = (input_size2 - filter_size + 2*padding)/stride + 1
        pool_output_size = int(filter_num* int(conv_output_size_h/2)* int(conv_output_size_w/2))

        self.params = {}
        rand = np.random.randn

        # He initialize
        self.params['cnn_W'] = rand(filter_num, input_dim[0], filter_size, filter_size) / np.sqrt(filter_num/2)
        self.params['cnn_b'] = np.zeros(filter_num)
        self.params['W1'] = rand(pool_output_size, hidden_size)/np.sqrt(pool_output_size/2)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = rand(hidden_size, output_size)/np.sqrt(hidden_size/2)
        self.params['b2'] = np.zeros(output_size)

        self.layers=[
                     Convolution(self.params['cnn_W'], self.params['cnn_b'], stride, padding),
                     ReLU(),
                     Pooling(pool_h=2, pool_w=2, stride=2),
                     FullyConnected(self.params['W1'], self.params['b1']),
                     ReLU(),
                     FullyConnected(self.params['W2'], self.params['b2']),
        ]
        self.loss_layer = ReluWithLoss()

        self.grads = []
        self.params = []
        for i in [0, 3, 5]:
            self.params += self.layers[i].params
            self.grads+=self.layers[i].grads

    def predict(self, x):
        x = np.array(x)
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def forward(self, x, t):
        x = np.array(x)
        t = np.array(t)
        x = self.predict(x)
        loss = self.loss_layer.forward(x, t)
        return loss

    def backward(self, dy=1):
        dy = self.loss_layer.backward(dy)

        for layer in reversed(self.layers):
            dy = layer.backward(dy)
        return dy
    
    def fit(self, train_X=None, train_y=None,learning_rate=0.01, epochs=10, batch_size=32, verbose=0):
        optimizer = Adam(learning_rate)

        data_size = train_X.shape[0]
        max_iters = data_size//batch_size

        for epoch in range(1, epochs+1):
            # shuffle train data
            idx = numpy.random.permutation(numpy.arange(data_size))
            x_data = train_X[idx]
            y_data = train_y[idx]

            epoch_loss = 0
            start_time = time.time()
            for iter in range(max_iters):
                batch_x = x_data[iter*batch_size:(iter+1)*batch_size]
                batch_y = y_data[iter*batch_size:(iter+1)*batch_size]

                loss = self.forward(batch_x, batch_y)
                self.backward()
                params, grads = self.params, self.grads
                optimizer.update(params, grads)

                epoch_loss += loss
            avg_loss = epoch_loss/max_iters

            if verbose:
                duration = start_time-time.time()
                print(f'epoch:{epoch}/{epochs}, 시간:{duration:.2f}[s], loss:{avg_loss:.5f}')

In [ ]:
# configuration setting
def model_config():
    # parameter for CNN Model
    filter_num = [30]
    filter_size = [3, 5]
    epochs = [30]
    batch_size = [32, 64]
    learning_rate = [0.1, 0.01]
    
    # create config data
    configs = []
    for i in filter_num:
        for j in filter_size:
            for k in epochs:
                for l in batch_size:
                    for m in learning_rate:
                        config = [i, j, k, l, m]
                        configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_filter, n_fsize, n_epochs, n_batch, learning_rate = config
    cnn_params = {'filter_num':n_filter, 'filter_size':n_fsize, 
                  'stride':1, 'padding':0}
    model = CnnModelReg(input_dim=(1, 24, 17),params=cnn_params)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    t = np.array(t)
    return np.mean((x-t)**2)

In [ ]:
# dataset
import pandas as pd
import numpy
import time
from datetime import datetime

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00492/Metro_Interstate_Traffic_Volume.csv.gz'
df = pd.read_csv(data_url,compression='gzip', index_col=7)
df.drop(columns='weather_description', inplace=True)
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,traffic_volume
date_time,,,,,,,
2012-10-02 09:00:00,None,288.28,0.0,0.0,40,Clouds,5545
2012-10-02 10:00:00,None,289.36,0.0,0.0,75,Clouds,4516
2012-10-02 11:00:00,None,289.58,0.0,0.0,90,Clouds,4767
2012-10-02 12:00:00,None,290.13,0.0,0.0,90,Clouds,5026
2012-10-02 13:00:00,None,291.14,0.0,0.0,75,Clouds,4918


In [ ]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.model_selection import TimeSeriesSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder

dataset = df.values
holiday_encoder = LabelEncoder()
weather_encoder = ColumnTransformer(
    [('1', OneHotEncoder(), [5])],
    remainder='passthrough'
)
dataset[:, 0] = holiday_encoder.fit_transform(dataset[:, 0])
dataset = weather_encoder.fit_transform(dataset)
dataset = pd.DataFrame(dataset)
cols = dataset.columns.tolist()
cols = cols[-1:]+cols[:-1]
dataset = dataset[cols]
dataset = dataset.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 17
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)


n_splits = 3
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs).split(new_df)
next(train_test_split)

configs = model_config()
history = []
best_error = []
i = 1

print('config : filter_num, filter_size, epochs, batch_size, learning_rate')

# nested cross validation for time series model
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = len(test_cv_X)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1, 1, n_inputs, n_features)
    val_X = val_X.reshape(-1, 1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1, 1, n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, 1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        predicted = model.predict(val_X)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f'error(rmse):{error.item():.2f}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)

    # outer
    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    predicted = selected_model.predict(test_cv_X)
    error = np.sqrt(MSE_metric(predicted, test_cv_y))
    best_error.append(error)

    # model eval
    print(f'best_model => error(rmse) : {error.item():.2f}, param:{configs[param]}')
    print()

config : filter_num, filter_size, epochs, batch_size, learning_rate
fold : 1/3
 == train [30, 3, 30, 32, 0.1] model ==  error(rmse):915.52
 == train [30, 3, 30, 32, 0.01] model ==  error(rmse):833.39
 == train [30, 3, 30, 64, 0.1] model ==  error(rmse):878.23
 == train [30, 3, 30, 64, 0.01] model ==  error(rmse):838.51
 == train [30, 5, 30, 32, 0.1] model ==  error(rmse):967.61
 == train [30, 5, 30, 32, 0.01] model ==  error(rmse):835.64
 == train [30, 5, 30, 64, 0.1] model ==  error(rmse):963.18
 == train [30, 5, 30, 64, 0.01] model ==  error(rmse):3798.49
best_model => error(rmse) : 965.33, param:[30, 3, 30, 32, 0.01]

fold : 2/3
 == train [30, 3, 30, 32, 0.1] model ==  error(rmse):880.81
 == train [30, 3, 30, 32, 0.01] model ==  error(rmse):841.35
 == train [30, 3, 30, 64, 0.1] model ==  error(rmse):926.20
 == train [30, 3, 30, 64, 0.01] model ==  error(rmse):930.87
 == train [30, 5, 30, 32, 0.1] model ==  error(rmse):1012.07
 == train [30, 5, 30, 32, 0.01] model ==  error(rmse):858

In [ ]:
model_evaluation = sum(best_error)
model_evaluation /= n_splits
print(f'evaluation [Mean RMSE] : {model_evaluation}')

evaluation [Mean RMSE] : 745.7493209648715


In [ ]:
predicted = selected_model.predict(test_cv_X)
print(f'MSE : {MSE_metric(predicted, test_cv_y)}')
print(f'RMSE : {np.sqrt(MSE_metric(predicted, test_cv_y))}')

def MAE_metric(x, t):
    t = np.array(t)
    return np.mean(numpy.abs(x-t))
print(f'MAE : {MAE_metric(predicted, test_cv_y)}')

MSE : 357871.14951647475
RMSE : 598.2233274592983
MAE : 454.25089911594443
